In [1]:
import os
import sys
sys.path.insert(0, '../gofher')

import itertools

from spin_parity import read_spin_parity_galaxies_label_from_csv
from sparcfire import read_sparcfire_galaxy_csv, get_ref_band_and_gofher_params
from panstarrs import create_panstarrs_csv, PANSTARRS_REF_BANDS_IN_ORDER, PANSTARRS_BANDS_IN_ORDER
from run_gofher_on_a_galaxy import run_panstarrs_with_sparcfire_center_only

In [2]:
path_to_input = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\original\\galaxies\\" #PANSTARRS
folder_name = "table3"

path_to_output = "C:\\Users\\school\\Desktop\\gofher_output_refactor"
output_folder_name = 'gofher_panstarrs_sparcfire_center_only_test'

visualize_gals=True
make_csv=True

def get_fits_path(name,band):
    """the file path of where existing fits files can be found"""
    return os.path.join(path_to_input,folder_name,name,"{}_{}.fits".format(name,band))

def get_galaxy_list():
    """the list of galaxy names to run on"""
    return os.listdir(os.path.join(path_to_input,folder_name))

def get_save_vis_path(name):
    """the file path specifying where to save the visualizitaion"""
    return os.path.join(path_to_output,output_folder_name,folder_name,"{}.png".format(name))

def get_csv_out_path():
    """the file path specifying where to save the ouput csv"""
    return os.path.join(path_to_output,output_folder_name,"{}.csv".format(folder_name))

def get_color_image_path(name):
    return os.path.join(path_to_input,folder_name,name,"{}_color.jfif".format(name))

def get_dark_side_csv_path():
    csv_path = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\table_info\\csv_format_of_table\\"
    return os.path.join(csv_path,"table_{}.csv".format(folder_name.strip()[-1]))

In [3]:
path_to_sparcfire_csv = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog\\original\\output_from_running\\SpArcFiRe\\{}\\galaxy.csv".format(folder_name)
sparcfire_galaxy_csv = read_sparcfire_galaxy_csv(path_to_sparcfire_csv)

dark_side_labels = read_spin_parity_galaxies_label_from_csv(get_dark_side_csv_path())
if folder_name == "table2" and "IC 2101" in dark_side_labels:
    dark_side_labels["IC2101"] = dark_side_labels["IC 2101"]

the_gals = []
i = 1
for name in sparcfire_galaxy_csv:
    print(i,name)
    i += 1
    try:
        sparcfire_bands = sparcfire_galaxy_csv[name]

        save_vis_path = ''
        if visualize_gals:
            save_vis_path=get_save_vis_path(name)

        paper_dark_side_label = dark_side_labels[name]
    
        the_gal = run_panstarrs_with_sparcfire_center_only(name, get_fits_path, sparcfire_bands, save_vis_path=save_vis_path,dark_side_label=paper_dark_side_label,color_image_path=get_color_image_path(name))
        the_gals.append(the_gal)
    except:
        print("Error when running on",name)

if make_csv:
    the_band_pairs = list(itertools.combinations(PANSTARRS_BANDS_IN_ORDER, 2)) #PANSTARRS
    #the_band_pairs = list(itertools.combinations(SDSS_BANDS_IN_ORDER, 2)) #SDSS
    create_panstarrs_csv(the_gals,the_band_pairs,get_csv_out_path())

1 IC1683
2 IC1755
3 IC 2101
4 IC5376
5 MCG-02-02-030
6 MCG-02-51-004
7 NGC1035
8 NGC1056
9 NGC1084
10 NGC1093
11 NGC1637
12 NGC1667
13 NGC169
14 NGC1961
15 NGC2280
16 NGC2347
17 NGC2410
18 NGC2639
19 NGC2683
20 NGC2715
21 NGC2742
22 NGC2775
23 NGC2782
24 NGC2903
25 NGC3160
26 NGC3175
27 NGC3227
28 NGC3310
29 NGC3312
30 NGC3368
31 NGC3646
32 NGC3672
33 NGC3815
34 NGC3900
35 NGC3949
36 NGC4062
37 NGC4293
38 NGC4310
39 NGC4414
40 NGC4424
41 NGC4451
42 NGC4580
43 NGC4625
44 NGC4632
45 NGC4666
46 NGC4698
47 NGC4826
48 NGC5005
49 NGC5303
50 NGC5395
51 NGC5426
52 NGC5448
53 NGC5635
54 NGC5678
55 NGC5692
56 NGC5949
57 NGC5980
58 NGC6015
59 NGC6106
60 NGC6132
61 NGC615
62 NGC6181
63 NGC6186
64 NGC6207
65 NGC6394
66 NGC6503
67 NGC6907
68 NGC6951
69 NGC6978
70 NGC7311
71 NGC7364
72 NGC7466
73 NGC7479
74 NGC7536
75 NGC7591
76 NGC7608
77 NGC7625
78 NGC7631
79 NGC772
80 NGC7738
81 NGC7782
82 NGC7787
83 NGC801
84 NGC871
85 NGC877
86 NGC949
87 NGC972
88 UGC10331
89 UGC10384
90 UGC1057
91 UGC10972
92 U